In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/receiving/season/2021/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Receptions', 'Receiving Targets', 'Receiving Yards', 
                 'Avg Yards per Reception', 'Receiving Touchdowns', 'Long Reception', '20+ Yard Receiving Plays',
                 'Receiving Yards per Game', 'Receiving Fumbles', 'Receiving Fumbles Lost', 'Receiving Yards After Catch', 'Receiving First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Receptions,Receiving Targets,Receiving Yards,Avg Yards per Reception,Receiving Touchdowns,Long Reception,20+ Yard Receiving Plays,Receiving Yards per Game,Receiving Fumbles,Receiving Fumbles Lost,Receiving Yards After Catch,Receiving First Downs
0,WR,17,145,191,"1,947",13.4,16,59,30,114.5,0,0,846,89
1,WR,17,108,167,"1,616",15.0,10,56,27,95.1,1,1,482,75
2,WR,16,123,169,"1,553",12.6,11,59,19,97.1,0,0,604,84
3,WR,17,81,128,"1,455",18.0,13,82,22,85.6,2,1,651,56
4,WR,16,77,121,"1,405",18.2,6,83,23,87.8,4,2,780,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,RB,12,1,2,-1,-1.0,0,0,0,-0.1,0,0,0,0
429,OT,4,1,1,-2,-2.0,0,-2,0,-0.5,0,0,-2,0
430,QB,16,1,1,-2,-2.0,0,0,0,-0.1,0,0,2,0
431,WR,15,1,4,-3,-3.0,0,0,0,-0.2,0,0,0,0


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2021
total_stats

,Rank,Name,Team,Position,Games Played,Receptions,Receiving Targets,Receiving Yards,Avg Yards per Reception,Receiving Touchdowns,Long Reception,20+ Yard Receiving Plays,Receiving Yards per Game,Receiving Fumbles,Receiving Fumbles Lost,Receiving Yards After Catch,Receiving First Downs,Year
0,1,Cooper Kupp,LAR,WR,17,145,191,"1,947",13.4,16,59,30,114.5,0,0,846,89,2021
1,2,Justin Jefferson,MIN,WR,17,108,167,"1,616",15.0,10,56,27,95.1,1,1,482,75,2021
2,3,Davante Adams,GB,WR,16,123,169,"1,553",12.6,11,59,19,97.1,0,0,604,84,2021
3,4,Ja'Marr Chase,CIN,WR,17,81,128,"1,455",18.0,13,82,22,85.6,2,1,651,56,2021
4,5,Deebo Samuel Sr.,SF,WR,16,77,121,"1,405",18.2,6,83,23,87.8,4,2,780,51,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,429,Larry Rountree III,LAC,RB,12,1,2,-1,-1.0,0,0,0,-0.1,0,0,0,0,2021
429,430,Mike Remmers,KC,OT,4,1,1,-2,-2.0,0,-2,0,-0.5,0,0,-2,0,2021
430,430,Taylor Heinicke,WSH,QB,16,1,1,-2,-2.0,0,0,0,-0.1,0,0,2,0,2021
431,432,Diontae Spencer,DEN,WR,15,1,4,-3,-3.0,0,0,0,-0.2,0,0,0,0,2021


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Receptions,Receiving Targets,Receiving Yards,Avg Yards per Reception,Receiving Touchdowns,Long Reception,20+ Yard Receiving Plays,Receiving Yards per Game,Receiving Fumbles,Receiving Fumbles Lost,Receiving Yards After Catch,Receiving First Downs,Year
0,1,Cooper Kupp,LAR,WR,17,145,191,"1,947",13.4,16,59,30,114.5,0,0,846,89,2021
1,2,Justin Jefferson,MIN,WR,17,108,167,"1,616",15.0,10,56,27,95.1,1,1,482,75,2021
2,3,Davante Adams,GB,WR,16,123,169,"1,553",12.6,11,59,19,97.1,0,0,604,84,2021
3,4,Ja'Marr Chase,CIN,WR,17,81,128,"1,455",18.0,13,82,22,85.6,2,1,651,56,2021
4,5,Deebo Samuel Sr.,SF,WR,16,77,121,"1,405",18.2,6,83,23,87.8,4,2,780,51,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,429,Larry Rountree III,LAC,RB,12,1,2,-1,-1.0,0,0,0,-0.1,0,0,0,0,2021
429,430,Mike Remmers,KC,OT,4,1,1,-2,-2.0,0,-2,0,-0.5,0,0,-2,0,2021
430,430,Taylor Heinicke,WSH,QB,16,1,1,-2,-2.0,0,0,0,-0.1,0,0,2,0,2021
431,432,Diontae Spencer,DEN,WR,15,1,4,-3,-3.0,0,0,0,-0.2,0,0,0,0,2021


In [8]:
total_stats.to_csv("Receiving_2021.csv", index = False)
print("Data saved to Receiving_2021.csv")

Data saved to Receiving_2021.csv
